In [9]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [12]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving one-shot-classification.zip to one-shot-classification.zip
User uploaded file "one-shot-classification.zip" with length 389068 bytes


In [13]:
!unzip Omniglot.zip
!unzip one-shot-classification.zip

Output hidden; open in https://colab.research.google.com to view.

code inspired by : https://github.com/Chtchou/Fellow_omniglot/blob/master/Fellowship_ai_One_shot_Omniglot.ipynb

# Triplet Loss Function

### Preprocessing dataset

In [14]:
import os
import glob

import tensorflow as tf
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, concatenate,AveragePooling2D, Lambda, Add
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
import h5py

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
PATH = "/content/Omniglot/images_background" #Training images path

PATH_TEST = "/content/Omniglot/images_evaluation" #Validation and test images path

In [16]:
alph_type = np.array(os.listdir(PATH)) #Give the different types of alphabet in our training data
print(f"List of the different alphabets:\n {alph_type}")
print(f"\nNumber of different alphabets: {len(alph_type)}")

List of the different alphabets:
 ['Tagalog' 'Korean' 'N_Ko' 'Anglo-Saxon_Futhorc' 'Latin'
 'Ojibwe_(Canadian_Aboriginal_Syllabics)'
 'Blackfoot_(Canadian_Aboriginal_Syllabics)' 'Arcadian'
 'Syriac_(Estrangelo)' 'Balinese' 'Japanese_(katakana)' 'Sanskrit'
 'Armenian' 'Asomtavruli_(Georgian)' 'Hebrew' 'Cyrillic' 'Gujarati'
 'Early_Aramaic' 'Grantha' 'Mkhedruli_(Georgian)' 'Malay_(Jawi_-_Arabic)'
 'Inuktitut_(Canadian_Aboriginal_Syllabics)' 'Bengali'
 'Japanese_(hiragana)' 'Futurama' 'Alphabet_of_the_Magi' 'Greek' 'Braille'
 'Burmese_(Myanmar)' 'Tifinagh']

Number of different alphabets: 30


In [17]:
alph_type_test = np.array(os.listdir(PATH_TEST)) #Give the different types of alphabet in our evaluation data
print(f"List of the different alphabets:\n {alph_type_test}")
print(f"\nNumber of different alphabets: {len(alph_type_test)}")

List of the different alphabets:
 ['Oriya' 'Malayalam' 'Gurmukhi' 'Atlantean' 'Ge_ez' 'Angelic' 'Tibetan'
 'Sylheti' 'Manipuri' 'ULOG' 'Avesta' 'Keble' 'Atemayar_Qelisayer'
 'Syriac_(Serto)' 'Glagolitic' 'Tengwar' 'Mongolian'
 'Old_Church_Slavonic_(Cyrillic)' 'Kannada' 'Aurek-Besh']

Number of different alphabets: 20


In [18]:
#First shuffle the alphabets
np.random.shuffle(alph_type)

alph_type_train = alph_type[:27]
alph_type_val = alph_type[27:]

print(f"List of the different alphabets in training set:\n {alph_type_train}")
print(f"\nNumber of different alphabets in training set: {len(alph_type_train)}")
print(f"\nList of the different alphabets in validation set:\n {alph_type_val}")
print(f"\nNumber of different alphabets in validation set: {len(alph_type_val)}")

List of the different alphabets in training set:
 ['Alphabet_of_the_Magi' 'Bengali' 'Tagalog' 'Early_Aramaic' 'Latin'
 'Cyrillic' 'Futurama' 'Sanskrit' 'Arcadian' 'Mkhedruli_(Georgian)'
 'Inuktitut_(Canadian_Aboriginal_Syllabics)' 'Anglo-Saxon_Futhorc'
 'Japanese_(hiragana)' 'Malay_(Jawi_-_Arabic)' 'Balinese'
 'Asomtavruli_(Georgian)' 'Greek' 'N_Ko' 'Braille' 'Gujarati'
 'Blackfoot_(Canadian_Aboriginal_Syllabics)' 'Syriac_(Estrangelo)'
 'Japanese_(katakana)' 'Tifinagh' 'Grantha' 'Korean'
 'Ojibwe_(Canadian_Aboriginal_Syllabics)']

Number of different alphabets in training set: 27

List of the different alphabets in validation set:
 ['Hebrew' 'Burmese_(Myanmar)' 'Armenian']

Number of different alphabets in validation set: 3


In [19]:
alph_num_char_train = {alphabet:len(os.listdir(f'{PATH}/{alphabet}')) for alphabet in alph_type_train}
num_of_char_train = alph_num_char_train.values()
print(f'\nThe maximum number of different character for one alphabet is {max(num_of_char_train)}')
print(f'The minimum number of different character for one alphabet is {min(num_of_char_train)}')
total_char_train = sum(num_of_char_train)
print(f'The total number of different character is {total_char_train}')


alph_num_char_val = {alphabet:len(os.listdir(f'{PATH}/{alphabet}')) for alphabet in alph_type_val}
num_of_char_val = alph_num_char_val.values()
print(f'\nThe maximum number of different character for one alphabet is {max(num_of_char_val)}')
print(f'The minimum number of different character for one alphabet is {min(num_of_char_val)}')
total_char_val = sum(num_of_char_val)
print(f'The total number of different character is {total_char_val}')


alph_num_char_test = {alphabet:len(os.listdir(f'{PATH_TEST}/{alphabet}')) for alphabet in alph_type_test}
num_of_char_test = alph_num_char_test.values()
print(f'\nThe maximum number of different character for one alphabet is {max(num_of_char_test)}')
print(f'The minimum number of different character for one alphabet is {min(num_of_char_test)}')
total_char_test = sum(num_of_char_test)
print(f'The total number of different character is {total_char_test}')



The maximum number of different character for one alphabet is 55
The minimum number of different character for one alphabet is 14
The total number of different character is 867

The maximum number of different character for one alphabet is 41
The minimum number of different character for one alphabet is 22
The total number of different character is 97

The maximum number of different character for one alphabet is 47
The minimum number of different character for one alphabet is 20
The total number of different character is 659


In [20]:
alph_num_char_ex_train = {}
for alphabet in alph_type_train:
    char_list_train = os.listdir(f'{PATH}/{alphabet}')
    for char in char_list_train:
        alph_num_char_ex_train[(alphabet,char)] = len(os.listdir(f'{PATH}/{alphabet}/{char}'))
num_of_example_train = alph_num_char_ex_train.values()
total_example_train = sum(num_of_example_train) 
print(min(num_of_example_train),max(num_of_example_train))
print(f'The total number of different pictures is {total_example_train}')


alph_num_char_ex_val = {}
for alphabet in alph_type_val:
    char_list_val = os.listdir(f'{PATH}/{alphabet}')
    for char in char_list_val:
        alph_num_char_ex_val[(alphabet,char)] = len(os.listdir(f'{PATH}/{alphabet}/{char}'))
num_of_example_val = alph_num_char_ex_val.values()
total_example_val = sum(num_of_example_val) 
print(min(num_of_example_val),max(num_of_example_val))
print(f'The total number of different pictures is {total_example_val}')

alph_num_char_ex_test = {}
for alphabet in alph_type_test:
    char_list_test = os.listdir(f'{PATH_TEST}/{alphabet}')
    for char in char_list_test:
        alph_num_char_ex_test[(alphabet,char)] = len(os.listdir(f'{PATH_TEST}/{alphabet}/{char}'))
num_of_example_test = alph_num_char_ex_test.values()
total_example_test = sum(num_of_example_test) 
print(min(num_of_example_test),max(num_of_example_test))
print(f'The total number of different pictures is {total_example_test}')

20 20
The total number of different pictures is 17340
20 20
The total number of different pictures is 1940
20 20
The total number of different pictures is 13180


In [21]:
alph_type_test = np.array(os.listdir(PATH_TEST)) #Give the different types of alphabet in our evaluation data
print(f"List of the different alphabets:\n {alph_type_test}")
print(f"\nNumber of different alphabets: {len(alph_type_test)}")

List of the different alphabets:
 ['Oriya' 'Malayalam' 'Gurmukhi' 'Atlantean' 'Ge_ez' 'Angelic' 'Tibetan'
 'Sylheti' 'Manipuri' 'ULOG' 'Avesta' 'Keble' 'Atemayar_Qelisayer'
 'Syriac_(Serto)' 'Glagolitic' 'Tengwar' 'Mongolian'
 'Old_Church_Slavonic_(Cyrillic)' 'Kannada' 'Aurek-Besh']

Number of different alphabets: 20


In [22]:
total_example_train = sum(num_of_example_train)
Y_train = np.array([i//20+1 for i in range(total_example_train)])
Y_train=Y_train.reshape(*Y_train.shape,1)
Y_train.shape

(17340, 1)

In [23]:
Y_val = np.array([i//20+1 for i in range(total_example_val)])
Y_val=Y_val.reshape(*Y_val.shape,1)
Y_val.shape

(1940, 1)

In [24]:
Y_test = np.array([i//20+1 for i in range(total_example_test)])
Y_test=Y_test.reshape(*Y_test.shape,1)
Y_test.shape

(13180, 1)

In [0]:
trainPath = []
for alphabet in alph_type_train:
    trainPath += glob.glob(f"{PATH}/{alphabet}/*/*.png")
trainPath[0:5]

valPath = []
for alphabet in alph_type_val:
    valPath += glob.glob(f"{PATH}/{alphabet}/*/*.png")
    
testPath = glob.glob(f"{PATH_TEST}/*/*/*.png")

In [27]:
SZ=64 #imagesize
channel_sz = 1 #number of channel

X_train = np.array([resize(mpimg.imread(i), (SZ,SZ), mode='edge') for i in trainPath])
X_train= X_train.reshape(*X_train.shape, channel_sz)
X_train.shape

(17340, 64, 64, 1)

In [28]:
X_val = np.array([resize(mpimg.imread(i), (SZ,SZ), mode='edge') for i in valPath])
X_val= X_val.reshape(*X_val.shape, channel_sz)
X_val.shape

(1940, 64, 64, 1)

In [29]:
X_test = np.array([mpimg.imread(i) for i in testPath])
X_test= X_test.reshape(*X_test.shape, channel_sz)
X_test.shape

(13180, 105, 105, 1)

In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 64)
            positive -- the encodings for the positive images, of shape (None, 64)
            negative -- the encodings for the negative images, of shape (None, 64)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[:,0:64], y_pred[:,64:128], y_pred[:,128:196]
    
    #Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1))
    #Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1))
    #Subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = tf.to_float(tf.greater(basic_loss,0))
    num_positive_triplets = tf.reduce_sum(valid_triplets)
    #Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0))/ alpha * 100
    
    return loss

In [0]:
def triplet_acc(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet accuracy
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 64)
            positive -- the encodings for the positive images, of shape (None, 64)
            negative -- the encodings for the negative images, of shape (None, 64)
    
    Returns:
    loss -- real number, value of the accuracy
    """
    
    anchor, positive, negative = y_pred[:,0:64], y_pred[:,64:128], y_pred[:,128:196]
    
    #Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    #Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    #Subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Count number of positive triplets (where triplet_loss > 0)
    hard_triplets = tf.to_float(tf.greater(basic_loss,alpha))
    num_hard_triplets = tf.reduce_sum(hard_triplets)
    #Count number of triplets
    all_triplets = tf.reduce_sum(tf.to_float(tf.greater(basic_loss,-10**10)))
    
    #Accuracy
    acc = 1 - num_hard_triplets/all_triplets
    
    return acc

In [0]:
def create_triplets(X, Y, num=1):
    """
    Create a list of valid triplets for each valid (anchor, positive) couple
    
    Arguments:
    X -- array of images
    Y -- array of classes corresponding to each image
    num -- number of negative images for each valid anchor and positive images - must be positive
           if num = 0, all possible valid couples are created
            For example : for one valid (A,P) couple we can select 'num' random N images. 
                          Thus 'num' triplets are created for this (A,P) couple
    
    Returns:
    (A,P,N) -- python tuple containing 3 arrays : 
            A -- the array for the anchor images, of shape (None, 64)
            P -- the array for the positive images, of shape (None, 64)
            N -- the array for the negative images, of shape (None, 64)
    """

    Y = Y.reshape(Y.shape[0],)
    A = []
    P = []
    N = []
    
    #We loop over all possible valid (A,P)
    for i in range(X.shape[0]):  
        list_pos = X[Y==Y[i]]
        for j in list_pos:
            #We provide a number 'num' of triplets for each valid (A,P)
            if num >=1:
                for k in range(num):
                    rand_num = np.random.randint(X.shape[0])
                    if np.array_equal(X[i],j) == False:
                        A.append(X[i])
                        P.append(j)
                        while np.array_equal(Y[rand_num], Y[i]):
                            rand_num = np.random.randint(X.shape[0])
                        N.append(X[rand_num])
            if num == 0:
                for k in range(X.shape[0]):
                    if np.array_equal(X[i],j) == False:
                        if np.array_equal(Y[i],Y[k]) == False:
                            A.append(X[i])
                            P.append(j)
                            N.append(X[k])
    
    A = np.array(A)
    P = np.array(P)
    N = np.array(N)
    
    return (A, P, N)

In [33]:
triplets_list_val = create_triplets(X_val, Y_val, 1)
print([i.shape for i in triplets_list_val])

[(36860, 64, 64, 1), (36860, 64, 64, 1), (36860, 64, 64, 1)]


In [0]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

def EmbModel(input_shape, l2):
    regul  = regularizers.l2(l2)
    kwargs = {'padding':'same', 'kernel_regularizer':regul,'kernel_initializer':'he_normal'}

    inp = Input(input_shape) # 64x64x1
    x   = Conv2D(32, (3,3), strides=1, **kwargs)(inp) #32x32x64
    x   = BatchNormalization()(x)
    x   = Activation('relu')(x)
    
    #Stage 0 / resblock 0
    y   = Conv2D(32, (1,1), strides=1, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(32, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(64, (1,1), strides=1, **kwargs)(y)
    
    x   = Conv2D(64, (1,1), strides=1, **kwargs) (x)
    x   = Add()([x,y])
    
    #Stage 0 / resblock 1
    x   = BatchNormalization()(x)
    x   = Activation('relu')(x)
    y   = Conv2D(32, (1,1), strides=1, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(32, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(64, (1,1), strides=1, **kwargs)(y)
    
    x   = Add()([x,y])
    
    #Stage 1 / resblock 0
    y   = Conv2D(64, (1,1), strides=2, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(64, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(128, (1,1), strides=1, **kwargs)(y)
    
    x   = Conv2D(128, (1,1), strides=2, **kwargs) (x)
    x   = Add()([x,y])
    
    #Stage 1 / resblock 1
    x   = BatchNormalization()(x)
    x   = Activation('relu')(x)
    y   = Conv2D(64, (1,1), strides=1, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(64, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(128, (1,1), strides=1, **kwargs)(y)
    
    x   = Add()([x,y])
    
    #Stage 2 / resblock 0
    y   = Conv2D(128, (1,1), strides=2, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(128, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(256, (1,1), strides=1, **kwargs)(y)
    
    x   = Conv2D(256, (1,1), strides=2, **kwargs) (x)
    x   = Add()([x,y])
    
    #Stage 2 / resblock 1
    x   = BatchNormalization()(x)
    x   = Activation('relu')(x)
    y   = Conv2D(128, (1,1), strides=1, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(128, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(256, (1,1), strides=1, **kwargs)(y)
    
    x   = Add()([x,y])
    
    #Stage 3 / resblock 0
    y   = Conv2D(256, (1,1), strides=2, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(256, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(512, (1,1), strides=1, **kwargs)(y)
    
    x   = Conv2D(512, (1,1), strides=2, **kwargs) (x)
    x   = Add()([x,y])
    
    #Stage 2 / resblock 1
    x   = BatchNormalization()(x)
    x   = Activation('relu')(x)
    y   = Conv2D(256, (1,1), strides=1, **kwargs)(x)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(256, (3,3), strides=1, **kwargs)(y)
    
    y   = BatchNormalization()(y)
    y   = Activation('relu')(y)
    y   = Conv2D(512, (1,1), strides=1, **kwargs)(y)
    
    x   = Add()([x,y])
    
    #Final
    x   = BatchNormalization()(x)
    x   = Activation('relu')(x)
    x   = AveragePooling2D(pool_size=8)(x)
    x   = Flatten()(x)
    x = Dense (64, activation ='tanh',kernel_initializer='he_normal') (x)

    
    ##Create model
    model = Model(inputs = inp, outputs = x, name='EmbModel')
    
    return model

In [0]:
#We define our global model
def global_model(size, channel_size=1, l2=1e-4):
    input_size = (size, size, channel_sz)                     

    A = Input(input_size)
    P = Input(input_size)
    N = Input(input_size)

    emb_model= EmbModel(input_size, l2)

    out_A = emb_model(A)
    out_P = emb_model(P)
    out_N = emb_model(N)

    y_pred = concatenate([out_A, out_P, out_N], axis =-1)

    full_model = Model(inputs = [A, P, N], outputs = y_pred)
    
    return full_model, emb_model

In [0]:
classification_model, emb_model = global_model(SZ,channel_sz, l2=1e-3)

In [37]:
classification_model.summary()
emb_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
EmbModel (Model)                (None, 64)           2401728     input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [0]:
from keras.preprocessing.image import random_rotation, random_shift, random_shear, random_zoom

def augmentation_pipeline(img_arr):
    img_arr = random_rotation(img_arr, 30, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    img_arr = random_shear(img_arr, intensity=0.2, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    img_arr = random_zoom(img_arr, zoom_range=(0.85, 1.15), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    img_arr = random_shift(img_arr, wrg=0.15, hrg=0.15,fill_mode='nearest')
    return img_arr

def batch_generator(X,Y, bs=32,hardmode = False):
    """
    Create a mini-batch generator
    
    Arguments:
    X -- array of images
    Y -- array of classes corresponding to each image
    bs -- size of the minibatch

    
    Returns:
    [A_batch, P_batch, N_batch], y_dummie) -- a mini-batch of size bs
    """
    Y = Y.reshape(Y.shape[0],)
    while True:
        #0. Initialize Anchor,Postive, Negative
        A_batch = []
        P_batch = []
        N_batch = []
        for i in range(bs) :
            #1.Choose a random Anchor Image
            rand_A_num = np.random.randint(X.shape[0])    

            #2.Choose a random Positive Image
            list_pos = X[Y==Y[rand_A_num]]                            #List of positive images
            rand_P_num = np.random.randint(len(list_pos))
            while np.array_equal(X[rand_A_num],list_pos[rand_P_num]):
                rand_P_num = np.random.randint(len(list_pos))
                

            #3.Choose a random Negative Image
            rand_N_num = np.random.randint(X.shape[0])
            while np.array_equal(Y[rand_N_num], Y[rand_A_num]):
                rand_A_num = np.random.randint(X.shape[0])
                
            A_augment = augmentation_pipeline(X[rand_A_num])
            P_augment = augmentation_pipeline(list_pos[rand_P_num])
            N_augment = augmentation_pipeline(X[rand_N_num])
                    
            A_batch.append(A_augment)
            P_batch.append(P_augment)
            N_batch.append(N_augment)
            
        A_batch = np.array(A_batch)
        P_batch = np.array(P_batch)
        N_batch = np.array(N_batch)
        
        y_dummie = np.zeros((len(A_batch),))
        
        yield ([A_batch, P_batch, N_batch], y_dummie)

### Training the model

In [0]:
#Full network 1/32 - Embeddings 64 - Maxout
A_val, P_val, N_val = triplets_list_val
zeros_vect_val = np.zeros(A_val[:,1,1].shape) 


batch_sz = 32

model_name = '/content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5'
print(model_name)
    
#We create a checkpoint to save the best model and add an early stopping
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience =10, verbose=1, mode='min')
lr_annealing = ReduceLROnPlateau(monitor='val_loss', patience = 2, epsilon=0.01, factor = 0.25, min_lr = 1e-7, verbose = 1, mode='min' )
callbacks_list = [checkpoint, early_stop, lr_annealing]
    

#We compile our model with the custom made triplet_loss
classification_model.compile(optimizer = 'adam', loss = triplet_loss, metrics =[triplet_acc])
    
classification_model.fit_generator(batch_generator(X_train, Y_train, batch_sz, hardmode=False), 
                                   steps_per_epoch = 800,
                                   epochs = 200,
                                   verbose = 2,
                                   validation_data = ([A_val, P_val, N_val], zeros_vect_val),
                                   callbacks = callbacks_list,)

/content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/200
 - 470s - loss: 35.2996 - triplet_acc: 0.9014 - val_loss: 31.6165 - val_triplet_acc: 0.9143

Epoch 00001: val_loss improved from inf to 31.61652, saving model to /content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5
Epoch 2/200
 - 446s - loss: 23.4048 - triplet_acc: 0.9491 - val_loss: 29.9092 - val_triplet_acc: 0.9237

Epoch 00002: val_loss improved from 31.61652 to 29.90925, saving model to /content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5
Epoch 3/200
 - 444s - loss: 19.4531 - triplet_acc: 0.9639 - val_loss: 22.1446 - val_triplet_acc: 0.9531

Epoch 00003: val_loss improved from 29.90925 to 22.14458, saving model to /content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5
Epoch 4/200
 - 444s - loss: 18.1805 - triplet_acc: 0.9670 - val_loss: 22.0729 - val_triplet_acc: 0.9524

Epoch 00004: val_loss improved from 22.14458 to 22.07288, saving model to /content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5
Epoch 5/200
 - 441s - loss: 1

### Testing the model

In [0]:
from keras.models import load_model
classification_model = load_model('/content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5')
global_loss=1e9
A_val, P_val, N_val = triplets_list_val
zeros_vect_val = np.zeros(A_val[:,1,1].shape) 

for weights in ['/content/gdrive/My Drive/Colab Notebooks/Saved Models/Triplet.h5']:
    classification_model.load_weights(weights)
    loss, acc = classification_model.evaluate([A_val, P_val, N_val], zeros_vect_val, batch_size = 32, verbose = 0)
    print(weights, loss, acc)
    if (loss < global_loss):
        model_name = weights
        global_loss = loss
        
classification_model.load_weights(model_name)
model_name, global_loss

In [0]:
import numpy as np
import copy
from scipy.ndimage import imread
from scipy.spatial.distance import cdist

# Parameters
nrun = 20 # number of classification runs
fname_label = 'class_labels.txt' # where class labels are stored for each run

def classification_run(main_folder,folder,f_load,f_cost,ftype='cost'):
	# Compute error rate for one run of one-shot classification
	#
	# Input
	#  folder : contains images for a run of one-shot classification
	#  f_load : itemA = f_load('file.png') should read in the image file and process it
	#  f_cost : f_cost(itemA,itemB) should compute similarity between two images, using output of f_load
	#  ftype  : 'cost' if small values from f_cost mean more similar, or 'score' if large values are more similar
	#
	# Output
	#  perror : percent errors (0 to 100% error)
	# 
	assert ((ftype=='cost') | (ftype=='score'))

	# get file names
	with open(main_folder+'/'+folder+'/'+fname_label) as f:
		content = f.read().splitlines()
	pairs = [line.split() for line in content]
	test_files  = [pair[0] for pair in pairs]
	train_files = [pair[1] for pair in pairs]
	answers_files = copy.copy(train_files)
	test_files.sort()
	train_files.sort()	
	ntrain = len(train_files)
	ntest = len(test_files)

	# load the images (and, if needed, extract features)
	train_items = [f_load(main_folder+'/'+f) for f in train_files]
	test_items  = [f_load(main_folder+'/'+f) for f in test_files ]

	# compute cost matrix
	costM = np.zeros((ntest,ntrain),float)
	for i in range(ntest):
		for c in range(ntrain):
			costM[i,c] = f_cost(test_items[i],train_items[c])
	if ftype == 'cost':
		YHAT = np.argmin(costM,axis=1)
	elif ftype == 'score':
		YHAT = np.argmax(costM,axis=1)
	else:
		assert False

	# compute the error rate
	correct = 0.0
	for i in range(ntest):
		if train_files[YHAT[i]] == answers_files[i]:
			correct += 1.0
	pcorrect = 100 * correct / ntest
	perror = 100 - pcorrect
	return perror

def ModHausdorffDistance(itemA,itemB):
	# Modified Hausdorff Distance
	#
	# Input
	#  itemA : [n x 2] coordinates of "inked" pixels
	#  itemB : [m x 2] coordinates of "inked" pixels
	#
	#  M.-P. Dubuisson, A. K. Jain (1994). A modified hausdorff distance for object matching.
	#  International Conference on Pattern Recognition, pp. 566-568.
	#
	D = cdist(itemA,itemB)
	mindist_A = D.min(axis=1)
	mindist_B = D.min(axis=0)
	mean_A = np.mean(mindist_A)
	mean_B = np.mean(mindist_B)
	return max(mean_A,mean_B)

def LoadImgAsPoints(fn):
	# Load image file and return coordinates of 'inked' pixels in the binary image
	# 
	# Output:
	#  D : [n x 2] rows are coordinates
	I = imread(fn,flatten=True)
	I = np.array(I,dtype=bool)
	I = np.logical_not(I)
	(row,col) = I.nonzero()
	D = np.array([row,col])
	D = np.transpose(D)
	D = D.astype(float)
	n = D.shape[0]
	mean = np.mean(D,axis=0)
	for i in range(n):
		D[i,:] = D[i,:] - mean
	return D

def CNNDistance(itemA, itemB):
    """
    Compute the euclidean distance between the embeddings of the two images. 
    The embeddings have been calculated by using the emb_model.
    
    Arguments:
    itemA -- array of images
    itemB -- array of images
    
    Returns:
    dist - euclidean distance
    """
    
    itemA = itemA.reshape(1, itemA.shape[0], itemA.shape[1], 1)
    itemB = itemB.reshape(1, itemB.shape[0], itemB.shape[1], 1)
    itemA_emb = emb_model.predict_on_batch(itemA)
    itemB_emb = emb_model.predict_on_batch(itemB)
    dist = np.linalg.norm(itemA_emb - itemB_emb) #2-norm by default
    return dist


def LoadImgAsArray(fn):
    """
    Load an image file, resize it and return an array
    
    Arguments:
    fn -- an image
    
    Returns:
    image -- an array of image 
    """
    picture = mpimg.imread(fn)
    image = resize(picture, (SZ,SZ), mode='edge')
    return image

In [45]:
#tanh32-4
print ('One-shot classification with Modified Hausdorff Distance versus embeddings triplet loss Distance')
perror = np.zeros(nrun)
perror_cnn = np.zeros(nrun)
for r in range(1,nrun+1):
	rs = str(r)
	if len(rs) == 1:
		rs = '0' + rs		
	perror[r-1] = classification_run('one-shot-classification','/run'+rs, LoadImgAsPoints, ModHausdorffDistance, 'cost')
	perror_cnn[r-1] = classification_run('one-shot-classification','run'+rs, LoadImgAsArray, CNNDistance, 'cost')
	print (" run " + str(r) + " ModHausdorffDistance" + "(error " + str(	perror[r-1] ) + "%)"+ "  -  Embeddings_triplet_loss_Distance" + " (error " + str(	perror_cnn[r-1] ) + "%)")		
total = np.mean(perror)
total_cnn = np.mean(perror_cnn)
print (" average error ModHausdorffDistance " + str(total) + "%" + "  average error Embeddings_triplet_loss_Distance " + str(total_cnn) + "%")

One-shot classification with Modified Hausdorff Distance versus embeddings triplet loss Distance


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


 run 1 ModHausdorffDistance(error 45.0%)  -  Embeddings_triplet_loss_Distance (error 0.0%)
 run 2 ModHausdorffDistance(error 35.0%)  -  Embeddings_triplet_loss_Distance (error 15.0%)
 run 3 ModHausdorffDistance(error 40.0%)  -  Embeddings_triplet_loss_Distance (error 5.0%)
 run 4 ModHausdorffDistance(error 25.0%)  -  Embeddings_triplet_loss_Distance (error 10.0%)
 run 5 ModHausdorffDistance(error 30.0%)  -  Embeddings_triplet_loss_Distance (error 10.0%)
 run 6 ModHausdorffDistance(error 15.0%)  -  Embeddings_triplet_loss_Distance (error 0.0%)
 run 7 ModHausdorffDistance(error 60.0%)  -  Embeddings_triplet_loss_Distance (error 0.0%)
 run 8 ModHausdorffDistance(error 35.0%)  -  Embeddings_triplet_loss_Distance (error 20.0%)
 run 9 ModHausdorffDistance(error 40.0%)  -  Embeddings_triplet_loss_Distance (error 10.0%)
 run 10 ModHausdorffDistance(error 55.0%)  -  Embeddings_triplet_loss_Distance (error 25.0%)
 run 11 ModHausdorffDistance(error 15.0%)  -  Embeddings_triplet_loss_Distance (err